In [1]:
# import relevant packages
from db_queries import get_outputs, get_ids
import pandas as pd
import numpy as np
from gbd_mapping import causes, risk_factors
from vivarium_inputs import get_measure
import gbd_mapping, vivarium_inputs, scipy.stats
from operator import attrgetter
from get_draws.api import get_draws
from math import sqrt, pi
import seaborn as sns
import matplotlib.pyplot as plt

# This notebook aims to investigate the probability that an individual *without anemia* will respond to iron supplementation.

In [8]:
# define sequelae IDs for all anemia and for iron-responsive anemias

mild_anemia = [144, 172, 177, 182, 206, 240, 525, 531, 537, 645, 648, 651, 654, 1016, 1024, 1032, 1057, 1061, 1065, 1069, 1079, 1089, 1099, 1106, 1120, 1373, 1385, 1397, 1421, 1433, 1445, 4952, 4955, 4976, 4985, 4988, 5009, 5018, 5027, 5036, 5051, 5063, 5075, 5087, 5099, 5111, 5123, 5225, 5228, 5249, 5252, 5273, 5276, 5393, 5567, 5579, 5606, 5627, 5678, 7202, 7214]
moderate_anemia = [145, 173, 178, 183, 207, 241, 526, 532, 538, 646, 649, 652, 655, 1017, 1025, 1033, 1058, 1062, 1066, 1070, 1080, 1090, 1100, 1107, 1121, 1376, 1388, 1400, 1424, 1436, 1448, 4958, 4961, 4979, 4991, 4994, 5012, 5021, 5030, 5039, 5054, 5066, 5078, 5090, 5102, 5114, 5126, 5219, 5222, 5243, 5246, 5267, 5270, 5396, 5570, 5582, 5609, 5630, 5681, 7205, 7217]
severe_anemia = [146, 174, 179, 184, 208, 242, 527, 533, 539, 647, 650, 653, 656, 1018, 1026, 1034, 1059, 1060, 1063, 1064, 1067, 1068, 1071, 1074, 1075, 1077, 1081, 1083, 1085, 1087, 1091, 1093, 1095, 1097, 1101, 1108, 1122, 1379, 1391, 1403, 1427, 1439, 1451, 4964, 4967, 4982, 4997, 5000, 5015, 5024, 5033, 5042, 5057, 5069, 5081, 5093, 5105, 5117, 5129, 5213, 5216, 5237, 5240, 5261, 5264, 5399, 5573, 5585, 5612, 5633, 5684, 7208, 7220]
mild_ira = [144, 172, 177, 182, 206, 240, 525, 537, 1016, 1024, 1032, 1106, 1373, 1385, 1397, 1421, 1433, 1445, 4952, 4955, 4976, 4985, 4988, 5009, 5225, 5228, 5249, 5252, 5273, 5276, 5393, 5567, 5579, 5627, 5678, 7202, 7214 ]
moderate_ira = [145, 173, 178, 183, 207, 241, 526, 538, 1017, 1025, 1033, 1107, 1376, 1388, 1400, 1424, 1436, 1448, 4958, 4961, 4979, 4991, 4994, 5012, 5219, 5222, 5243, 5246, 5267, 5270, 5396, 5570, 5582, 5630, 5681, 7205, 7217]
severe_ira = [146, 174, 179, 184, 208, 242, 527, 539, 1018, 1026, 1034, 1108, 1379, 1391, 1403, 1427, 1439, 1451, 4964, 4967, 4982, 4997, 5000, 5015, 5213, 5216, 5237, 5240, 5261, 5264, 5399, 5573, 5585, 5633, 5684, 7208, 7220]
total_anemia = mild_anemia + moderate_anemia + severe_anemia
total_ira = mild_ira + moderate_ira + severe_ira

In [9]:
# define sequelae IDs for non iron responsive anemias
mild_nira = []
moderate_nira = []
severe_nira = []

for i in mild_anemia:
    if i not in mild_ira:
        mild_nira.append(i)
for i in moderate_anemia:
    if i not in moderate_ira:
        moderate_nira.append(i)
for i in severe_anemia:
    if i not in severe_ira:
        severe_nira.append(i)
total_nira = mild_nira + moderate_nira + severe_nira

In [125]:
# call sequela IDs and names
s = get_ids('sequela')

In [171]:
# prevalence of all anemia (India, post-natal, 2017)
all_prevalence = get_outputs(gbd_round_id=5, year_id=2017, sex_id=3,
                             location_id=163, age_group_id=4, topic='sequela',
                             sequela_id=total_anemia, measure_id=5, metric_id=3)
all_prevalence = all_prevalence['val'].sum()
all_prevalence

0.6736174893247817

In [127]:
# prevalence of iron responsive anemia (India, post-natal, 2017)
ira_prevalence = get_outputs(gbd_round_id=5, year_id=2017, sex_id=3,
                             location_id=163, age_group_id=4, topic='sequela',
                             sequela_id=total_ira, measure_id=5, metric_id=3)
ira_prevalence = ira_prevalence['val'].sum()
ira_prevalence

0.6352051114906149

In [128]:
# prevalence of non iron responsive anemia (India, post-natal, 2017)
nira_prevalence = get_outputs(gbd_round_id=5, year_id=2017, sex_id=3,
                             location_id=163, age_group_id=4, topic='sequela',
                             sequela_id=total_nira, measure_id=5, metric_id=3)
nira_prevalence = nira_prevalence['val'].sum()
nira_prevalence

0.038412377834166934

In [129]:
# probability of iron-responsiveness AMONG ANEMIC INDIVIDUALS
ira_prevalence/all_prevalence

0.9429759790342283

## Notably, there are three GBD causes that cause anemia and do not respond to iron supplementation, including

- HIV/AIDs

- Malaria

- Hemoblonopathies/hemolytic anemias

These causes will be referred to as "non-responsive causes"

In [172]:
# prevalence of non-responsive causes (India, post-natal, 2017)
nr_prevalence = get_outputs(gbd_round_id=5, year_id=2017, sex_id=3,
                             location_id=163, age_group_id=4, topic='cause',
                             cause_id=[613,298,345], measure_id=5, metric_id=3)
nr_prevalence = nr_prevalence['val'].sum()
nr_prevalence

0.368214619696644

In [181]:
no_anemia_prevalence = (1 - all_prevalence)
no_anemia_prevalence

0.3263825106752183

In [182]:
non_responsive_causes_without_anemia_prevalence = (nr_prevalence - nira_prevalence)
non_responsive_causes_without_anemia_prevalence

0.32980224186247703

In [183]:
no_anemia_prevalence - non_responsive_causes_without_anemia_prevalence

# Notably, the prevalence of those with non responsive causes and no anemia 
# is reportedly larger than the prevalence of those without anemia in general

# Why is this??

-0.0034197311872587455

In [198]:
# there are severeal sequela among the non-responsive causes that are "asymptomatic" or "without anemia"
without_anemia = [5060,5072,5096,1113,1114,1115,1072,1073,1076,1082,1084,1086,1088,1092,1094,1096,1098,5084,5108,5120,5132]
asymptomatic_or_without_anemia = [1126,1051,1052,1053,5060,5072,5096,1113,1114,1115,1072,1073,1076,1082,1084,1086,1087,1088,
 1092,1094,1096,1098,1050,1056,5615,5084,5108,5120,5132]
asymptomatic = []
for i in asymptomatic_or_without_anemia:
    if i not in without_anemia:
        asymptomatic.append(i)
asymptomatic

[1126, 1051, 1052, 1053, 1087, 1050, 1056, 5615]

In [186]:
nr_asymp_or_without_prev = get_outputs(gbd_round_id=5, year_id=2017, sex_id=3,
                             location_id=163, age_group_id=4, topic='sequela',
                             sequela_id=asymptomatic_or_without_anemia, measure_id=5, metric_id=3)
nr_asymp_or_without_prev = nr_asymp_or_without_prev['val'].sum()
nr_asymp_or_without_prev

# NOTE: this is slightly higher than the prevalence of the parent causes of nr_prevalence defined above
# this is likely due to como adjustments

0.385952868092631

In [194]:
nr_asymptomatic_prev = get_outputs(gbd_round_id=5, year_id=2017, sex_id=3,
                             location_id=163, age_group_id=4, topic='sequela',
                             sequela_id=asymptomatic, measure_id=5, metric_id=3)
nr_asymptomatic_prev = nr_asymptomatic_prev['val'].sum()
nr_asymptomatic_prev

0.38580994836841004

In [187]:
nr_without_anemia_prev = get_outputs(gbd_round_id=5, year_id=2017, sex_id=3,
                             location_id=163, age_group_id=4, topic='sequela',
                             sequela_id=without_anemia, measure_id=5, metric_id=3)
nr_without_anemia_prev = nr_without_anemia_prev['val'].sum()
nr_without_anemia_prev

0.00014291972422100872

Notably, the prevalence of asymptomatic non-responsive causes is much higher than the prevalence of non-responsive causes without anemia

In [195]:
nr_asymptomatic = get_outputs(gbd_round_id=5, year_id=2017, sex_id=3,
                             location_id=163, age_group_id=4, topic='sequela',
                             sequela_id=asymptomatic, measure_id=5, metric_id=3)
nr_asymptomatic

,age_group_id,location_id,measure_id,metric_id,sequela_id,sex_id,year_id,age_group_name,expected,location_name,measure_name,metric_name,sequela_name,sex,val,upper,lower
0,4,163,5,3,1050,3,2017,Post Neonatal,False,India,Prevalence,Rate,Asymptomatic B-thalassemia trait,Both,0.010717,0.011344,0.010093
1,4,163,5,3,1051,3,2017,Post Neonatal,False,India,Prevalence,Rate,Asymptomatic hemoglobin E trait,Both,0.010462,0.012138,0.008917
2,4,163,5,3,1052,3,2017,Post Neonatal,False,India,Prevalence,Rate,Asymptomatic sickle cell trait,Both,0.132145,0.155107,0.112623
3,4,163,5,3,1053,3,2017,Post Neonatal,False,India,Prevalence,Rate,Asymptomatic hemizygous G6PD trait,Both,0.156794,0.158876,0.154579
4,4,163,5,3,1056,3,2017,Post Neonatal,False,India,Prevalence,Rate,Asymptomatic malaria parasitemia (PfPR),Both,0.001014,0.001112,0.000924
5,4,163,5,3,1087,3,2017,Post Neonatal,False,India,Prevalence,Rate,"Hemoglobin SC disease, with vaso-occlusive cri...",Both,0.000000,0.000000,0.000000
6,4,163,5,3,1126,3,2017,Post Neonatal,False,India,Prevalence,Rate,Asymptomatic G6PD deficiency,Both,0.071588,0.074107,0.068912
7,4,163,5,3,5615,3,2017,Post Neonatal,False,India,Prevalence,Rate,Asymptomatic malaria vivax (PvPR),Both,0.003090,0.003201,0.002986


This is primarily driven by asymptomatic sickle cell trait and asymptomatic hemozygous G6PD trait/deficiency. Notably, asymptomatic individuals with these conditions generally remain asymptomatic. We will make the same assumption about other asymptomatic genetic disorders and malaria.

Therefore, if we assume that asymptomatic individuals will remain asymptomatic and exclude them from our analysis and evaluate the prevalence of individuals afflicated with non iron responsive causes *without anemia*...

In [202]:
nr_without_anemia_prev

0.00014291972422100872

In [203]:
# probability of non-iron responsiveness without anemia given no anemia
nr_without_anemia_prev / no_anemia_prevalence

0.0004378902653985264

We see that the prevalence is very small for this location and age group. Therefore, we can assume that individuals who are border-line anemic and *at risk* of becoming anemic due to a non iron responsive cause (non-asymptomatic individuals) represent a neglible proportion of individuals without anemia and can be ignored. **Therefore, we assume that all individuals without anemia are iron responsive.**